In [1]:
import earthaccess
import pandas as pd

In [2]:
auth = earthaccess.login(strategy="interactive", persist=True)

In [77]:
# earthaccess.search_data(
#     short_name=[
#         # "M2I1NXINT", #ok
#         # "M2I3NVASM", #ok
#         # "M2T1NXRAD", #ok
#         # "M2T1NXFLX", #ok
#         # "M2T1NXLND", #ok
#         # "M2I1NXASM", #ok
#     ],
#     bounding_box=(6.488200, 14.209860, 6.488300, 14.209960),
#     temporal=(f"2021-01-01T00:00:00", f"2021-12-31T23:59:59"),
# )

Granules found: 0


[]

In [ ]:
# datasets = earthaccess.search_datasets(keyword="NDVI")

# # # Display the short names and titles of the retrieved datasets
# # for dataset in datasets:
# #     print(f"Short Name: {dataset['ShortName']}, Title: {dataset['EntryTitle']}")


Datasets found: 175


# Code

In [35]:
df_train = pd.read_csv('locust_breeding/records/train.csv')
df_train

,Unnamed: 0,x,y,date,label,generated,CAT,year
0,1110,6.488200,14.209860,2020-06-09,0.0,1,PseudoAbsence,2020
1,4771,26.388200,-30.390140,2020-10-24,0.0,1,PseudoAbsence,2020
2,2981,20.188200,-34.290140,2020-11-28,0.0,1,PseudoAbsence,2020
3,5523,66.688200,36.409860,2020-06-08,0.0,1,PseudoAbsence,2020
4,974,23.288200,1.809860,2020-11-24,0.0,1,PseudoAbsence,2020
...,...,...,...,...,...,...,...,...
4471,12378,39.331030,15.810600,2020-01-04,1.0,0,NaN,2020
4472,11496,65.528333,28.261111,2020-03-15,1.0,0,NaN,2020
4473,10108,74.873889,28.605278,2020-08-09,1.0,0,NaN,2020
4474,8969,70.803665,32.588367,2020-06-08,1.0,0,NaN,2020


In [36]:
df_train.x.min(), df_train.x.max(), df_train.y.min(), df_train.y.max() 

(-16.811799999999998, 77.5882, -37.090140000000005, 37.50986)

In [34]:
import sys
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import getpass

In [37]:
# STEP 2
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

In [38]:
# STEP 3
# This method POSTs formatted JSON WSP requests to the GES DISC endpoint URL
# It is created for convenience since this task will be repeated more than once
def get_http_data(request):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty %s request' % response['methodname'])
        sys.exit(1)
    return response

In [50]:
# STEP 4
# Define the parameters for the second subset example
product = 'M2T1NXSLV_5.12.4'
varNames =['CDH', 'CDM', 'CDQ', 'QSTAR']
minlon = 6.488200 - 1
maxlon = 6.488200 + 1
minlat = 14.209860 - 1
maxlat = 14.209860 + 1
begTime = '2020-06-08'
endTime = '2020-06-09'

diurnalAggregation = '1'
interp = 'remapbil'
destGrid = 'cfsr0.5a'

In [51]:
# STEP 5
# Construct JSON WSP request for API method: subset
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True,
        'diurnalAggregation': diurnalAggregation,
        'mapping': interp,
        'grid'  : destGrid,
        'data': [{'datasetId': product,
                  'variable' : varNames[0]
                 },
                 {'datasetId': product,
                  'variable' : varNames[1]
                 },
                 {'datasetId': product,
                  'variable' : varNames[2]
                 }]
    }
}

In [52]:
# STEP 6
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

Job ID: 67a148b9b938d51db82265bf
Job status: Accepted


In [53]:
# STEP 7
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Job status: Succeeded (100% complete)
Job Finished:  Complete (M2T1NXSLV_5.12.4)


In [55]:
# STEP 8 (Plan A - preferred)
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 3 out of 3 expected items


In [56]:
# Sort the results into documents and URLs

docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
# print('\nDocumentation:')
#for item in docs : print(item['label']+': '+item['link'])

In [57]:
# STEP 10 
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn, "is downloaded")
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
        
print('Downloading is done and find the downloaded files in your current working directory')


HTTP_services output:


AttributeError: 'Response' object has no attribute 'status'

In [73]:
import earthaccess
import xarray as xr

# This will work if Earthdata prerequisite files have already been generated
auth = earthaccess.login()

# We recommend only streaming a few granules at a time, as some collections can be quite large
results = earthaccess.search_data(
    doi="10.5067/7MCPBJ41Y0K6",
    temporal=('2020-06-08', '2020-06-09'), # This will stream one granule, but can be edited for a longer temporal extent
    bounding_box=(-180, 0, 180, 90)
)


downloaded_files = earthaccess.download(
    results,
    local_path='.', # Change this string to download to a different path
)

Granules found: 2
 Getting 2 granules, approx download size: 0.74 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

File MERRA2_400.tavg1_2d_flx_Nx.20200608.nc4 already downloaded


PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

In [96]:
ds = xr.open_dataset(downloaded_files[0]) # Opens the two granules and concatenates them along the time dimension
ds

<xarray.Dataset> Size: 918MB
Dimensions:      (lon: 576, lat: 361, time: 24)
Coordinates:
  * lon          (lon) float64 5kB -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * lat          (lat) float64 3kB -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * time         (time) datetime64[ns] 192B 2020-06-08T00:30:00 ... 2020-06-0...
Data variables: (12/46)
    BSTAR        (time, lat, lon) float32 20MB ...
    CDH          (time, lat, lon) float32 20MB ...
    CDM          (time, lat, lon) float32 20MB ...
    CDQ          (time, lat, lon) float32 20MB ...
    CN           (time, lat, lon) float32 20MB ...
    DISPH        (time, lat, lon) float32 20MB ...
    ...           ...
    TSTAR        (time, lat, lon) float32 20MB ...
    ULML         (time, lat, lon) float32 20MB ...
    USTAR        (time, lat, lon) float32 20MB ...
    VLML         (time, lat, lon) float32 20MB ...
    Z0H          (time, lat, lon) float32 20MB ...
    Z0M          (time, lat, lon) float32 20MB ...
Attributes: (12/30)
    History:                           Original file generated: Thu Jun 18 20...
    Comment:                           GMAO filename: d5124_m2_jan10.tavg1_2d...
    Filename:                          MERRA2_400.tavg1_2d_flx_Nx.20200608.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    Contact:                           http://gmao.gsfc.nasa.gov
    identifier_product_doi:            10.5067/7MCPBJ41Y0K6
    RangeBeginningDate:                2020-06-08
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2020-06-08
    RangeEndingTime:                   23:59:59.000000

In [17]:
import xarray as xr

# Subsetting a .ncml file URL:
#URL = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/thredds/dodsC/MERRA2_aggregation/M2T1NXSLV.5.12.4/M2T1NXSLV.5.12.4_Aggregation_1980.ncml'
URL = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/M2T1NXFLX.5.12.4/2020/06/MERRA2_400.tavg1_2d_flx_Nx.20200609.nc4?CN[0:1:23][0:1:360][0:1:575],BSTAR[0:1:23][0:1:360][0:1:575],QSTAR[0:1:23][0:1:360][0:1:575]'

lat_slice = slice(41, 43)
lon_slice = slice(-89, -87)
#time_slice = slice('2020-06-08', '2020-06-09') # Create a 3-day subset

try:
    # Access the THREDDS server ncml file, which will handle subset requests
    ds = xr.open_dataset(URL)

    # Send the subset request and stream data to the notebook using the bounding box and time slice
    ds_subset = ds.sel(lat=lat_slice,lon=lon_slice)
    print(ds_subset['QSTAR'])
except OSError as e:
    print('Error', e)
    print('Please check that your .dodsrc files are in their correct locations, or that your .netrc file has the correct username and password.')
    raise

<xarray.DataArray 'QSTAR' (time: 24, lat: 2, lon: 2)> Size: 384B
[96 values with dtype=float32]
Dimensions without coordinates: time, lat, lon
Attributes:
    long_name:       surface_moisture_scale
    units:           kg kg-1
    fmissing_value:  1000000000000000.0
    standard_name:   surface_moisture_scale
    vmax:            1000000000000000.0
    vmin:            -1000000000000000.0
    valid_range:     [-1.e+15  1.e+15]
    origname:        QSTAR
    fullnamepath:    /QSTAR


In [20]:
ds.to_netcdf('test2.nc')

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.
syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: HTTP^ Basic: Access denied.


RuntimeError: NetCDF: Access failure

In [32]:
import earthaccess

# 1. Authenticate (handles NASA EDL login)
earthaccess.login()

# 2. Search for MERRA-2 M2T1NXFLX data
results = earthaccess.search_data(
    short_name="M2T1NXFLX",  # Official dataset short name
    temporal=("2020-05-09", "2020-06-09"),  # Date range (start, end)
    count=10,  # Number of granules to retrieve (-1 for all)
    cloud_hosted=True  # Access cloud-stored data
)

# 3. Download NetCDF files
files = earthaccess.download(results, "./merra_data")

Granules found: 32
 Getting 10 granules, approx download size: 3.73 GB


QUEUEING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

File MERRA2_400.tavg1_2d_flx_Nx.20200510.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200511.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200509.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200514.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200513.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200512.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200515.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200516.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200517.nc4 already downloaded
File MERRA2_400.tavg1_2d_flx_Nx.20200518.nc4 already downloaded


PROCESSING TASKS | :   0%|          | 0/10 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
bounding_box=(-180, -90, 180, 90),  # W,S,E,N coordinates
version="5.12.4",  # Specific processing version
variables=["EVAP", "EFLUX"]  # Select specific variables

# OLD

In [9]:
import xarray as xr

# Specify 1980-01-01 Cloud OPeNDAP URL
#dataset_url = 'https://opendap.earthdata.nasa.gov/collections/C1276812863-GES_DISC/granules/M2T1NXSLV.5.12.4%3AMERRA2_100.tavg1_2d_slv_Nx.19800101.nc4'
dataset_path = 'MERRA2_100.tavg1_2d_slv_Nx.19800104.SUB.nc'

try:
    # Load dataset object and metadata, but don't open the values yet
    # NOTE: When opening HDF files, the group to be accessed must be specified with the "group=" parameter. 
    #       E.g., for GPM IMERG, group="Grid" must be entered or an error will occur
    ds = xr.open_dataset(dataset_path)
except OSError as e:
    print('Error', e)
    print('Please check that your .dodsrc files are in their correct locations, or that your .netrc file has the correct username and password.')
    raise

# Define latitude and longitude bounds for CONUS
lat_min, lat_max = 14.209860 - 1 , 14.209860 + 1 # Latitude bounds
lon_min, lon_max = 6.488200 - 1, 6.488200 + 1 # Longitude bounds

# Subset the dataset based on lat/lon bounds, which is performed server-side
ds_conus = ds.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
ds_conus
# Plot the first timestep of the 2-meter temperature variable from the subsetted dataset
#ds_conus['T2M'].isel(time=0).plot(cmap='coolwarm')


<xarray.Dataset> Size: 56B
Dimensions:    (time: 1, bnds: 2, lon: 4, lat: 0)
Coordinates:
  * time       (time) datetime64[ns] 8B 1980-01-04T12:00:00
  * lon        (lon) float64 32B 5.5 6.0 6.5 7.0
  * lat        (lat) float64 0B 
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 16B ...
    TQI        (time, lat, lon) float32 0B ...
    TQL        (time, lat, lon) float32 0B ...
    TQV        (time, lat, lon) float32 0B ...
Attributes: (12/32)
    CDI:                               Climate Data Interface version 1.9.8 (...
    Conventions:                       CF-1
    History:                           Original file generated: Sat May 31 21...
    Comment:                           GMAO filename: d5124_m2_jan79.tavg1_2d...
    Filename:                          MERRA2_100.tavg1_2d_slv_Nx.19800104.nc4
    Institution:                       NASA Global Modeling and Assimilation ...
    ...                                ...
    RangeBeginningDate:                1980-01-04
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   1980-01-04
    RangeEndingTime:                   23:59:59.000000
    history_L34RS:                     'Created by L34RS v1.4.4 @ NASA GES DI...
    CDO:                               Climate Data Operators version 1.9.8 (...

In [20]:
ds['lon'].values #.sel(lat=slice(-90, 90))

array([  0. ,   0.5,   1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,
         4.5,   5. ,   5.5,   6. ,   6.5,   7. ,   7.5,   8. ,   8.5,
         9. ,   9.5,  10. ,  10.5,  11. ,  11.5,  12. ,  12.5,  13. ,
        13.5,  14. ,  14.5,  15. ,  15.5,  16. ,  16.5,  17. ,  17.5,
        18. ,  18.5,  19. ,  19.5,  20. ,  20.5,  21. ,  21.5,  22. ,
        22.5,  23. ,  23.5,  24. ,  24.5,  25. ,  25.5,  26. ,  26.5,
        27. ,  27.5,  28. ,  28.5,  29. ,  29.5,  30. ,  30.5,  31. ,
        31.5,  32. ,  32.5,  33. ,  33.5,  34. ,  34.5,  35. ,  35.5,
        36. ,  36.5,  37. ,  37.5,  38. ,  38.5,  39. ,  39.5,  40. ,
        40.5,  41. ,  41.5,  42. ,  42.5,  43. ,  43.5,  44. ,  44.5,
        45. ,  45.5,  46. ,  46.5,  47. ,  47.5,  48. ,  48.5,  49. ,
        49.5,  50. ,  50.5,  51. ,  51.5,  52. ,  52.5,  53. ,  53.5,
        54. ,  54.5,  55. ,  55.5,  56. ,  56.5,  57. ,  57.5,  58. ,
        58.5,  59. ,  59.5,  60. ,  60.5,  61. ,  61.5,  62. ,  62.5,
        63. ,  63.5,

# Model

In [28]:
from huggingface_hub import hf_hub_download


In [29]:
config_path = hf_hub_download(
    repo_id="ibm-granite/granite-geospatial-wxc-downscaling",
    filename="config.yaml",
    local_dir="./"
)

config.yaml:   0%|          | 0.00/2.99k [00:00<?, ?B/s]

In [26]:
from transformers import AutoModelForPreTraining
import torch

# Load pretrained model
model = AutoModelForPreTraining.from_pretrained("ibm-nasa-geospatial/Prithvi-WxC-1.0-2300M")

ValueError: Unrecognized model in ibm-nasa-geospatial/Prithvi-WxC-1.0-2300M. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electra, emu3, encodec, encoder-decoder, ernie, ernie_m, esm